In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.widgets as widgets
from astropy.io import fits
from astropy.stats import sigma_clipped_stats
from astropy.visualization import ImageNormalize, ZScaleInterval, LogStretch, LinearStretch
from photutils.detection import find_peaks
import argparse
import os


In [2]:
from psf_photometry_mplw import get_frame_info

In [3]:
path = '/Users/bencapistrant/Documents/GitHub/rho_data_reduction_pipeline/data/2025-02-17/Reduced/'

In [18]:
uncert_df = pd.read_csv(path+'Uncertainties.csv')
uncert_df

FileNotFoundError: [Errno 2] No such file or directory: '/Users/bencapistrant/Documents/GitHub/rho_data_reduction_pipeline/data/2025-02-17/Reduced/Uncertainties.csv'

In [13]:
# Split each row in the first column by space and expand into separate columns
uncert_split = uncert_df[0].str.split(' ', n=1, expand=True)
uncert_split.columns = ['Type', 'Value']
uncert_split

,Type,Value
0,Read_Noise,42.59765251748035
1,Dark_Current_150.0s,9.886015649883465
2,Flat_R_Noise,0.05301435201167102


In [ ]:
# pd.DataFrame(columns=["Read_Noise", "Dark_Current", "Flat_Noise"])

,Read_Noise,Dark_Current,Flat_Noise


In [14]:
def get_frame_info(directories):
    """
    Extracts information from FITS file headers for reduced frames across multiple directories.
    Parameters:
    directories (list of str): List of directories containing reduced FITS frames.

    Returns:
    pandas.DataFrame: A DataFrame with the following columns:
    - 'Directory': Directory name.
    - 'File': File name.
    - 'Object': Object name (header keyword 'OBJECT').
    - 'Date-Obs': Observation date and time (header keyword 'DATE-OBS').
    - 'Filter': Filter used (header keyword 'FILTER').
    - 'Exptime': Exposure time in seconds (header keyword 'EXPTIME').
    """
    # Define lists to store extracted data
    directories_list = []
    file_list = []
    objects = []
    dates = []
    filters = []
    exposure_times = []

    # Iterate through all directories
    for directory in directories:

        # Get all FITS files in the directory
        fits_files = [f for f in os.listdir(directory) if f.endswith('.fits')]

        for file in fits_files:
            try:
                # Read the FITS header
                header = fits.getheader(os.path.join(directory, file))

                # Extract relevant header information
                directories_list.append(directory)
                file_list.append(file)
                objects.append(header.get('OBJECT', 'Unknown'))
                dates.append(header.get('DATE-OBS', 'Unknown'))
                filters.append(header.get('FILTER', 'Unknown'))
                exposure_times.append(header.get('EXPTIME', 'Unknown'))

            except Exception as e:
                print(f"Error processing file {file} in {directory}: {e}")
                continue
        uncert = pd.read_csv(directory + 'Uncertainties.csv', header=None)
        uncertainties = uncert[0].str.split(' ', n=1, expand=True)
        uncertainties.columns = ['Type', 'Value']
        # uncert_split
    # Create a DataFrame with the extracted information
    reduced_frame_info = pd.DataFrame({'Directory': directories_list,
                                       'File': file_list,
                                       'Object': objects,
                                       'Date-Obs': dates,
                                       'Filter': filters,
                                       'Exptime': exposure_times})

    return reduced_frame_info, uncertainties

In [16]:
frame, uncertainty = get_frame_info([path])

In [17]:
frame

,Directory,File,Object,Date-Obs,Filter,Exptime
0,/Users/bencapistrant/Documents/GitHub/rho_data...,WASP36B.2025-02-17.0132_reduced.fits,WASP-36B,2025-02-18T01:44:30.345,R,150.0
1,/Users/bencapistrant/Documents/GitHub/rho_data...,WASP36B.2025-02-17.0135_reduced.fits,WASP-36B,2025-02-18T01:52:14.432,R,150.0
2,/Users/bencapistrant/Documents/GitHub/rho_data...,WASP36B.2025-02-17.0160_reduced.fits,WASP-36B,2025-02-18T02:57:57.963,R,150.0
3,/Users/bencapistrant/Documents/GitHub/rho_data...,WASP36B.2025-02-17.0175_reduced.fits,WASP-36B,2025-02-18T03:36:17.242,R,150.0
4,/Users/bencapistrant/Documents/GitHub/rho_data...,WASP36B.2025-02-17.0172_reduced.fits,WASP-36B,2025-02-18T03:28:39.035,R,150.0
5,/Users/bencapistrant/Documents/GitHub/rho_data...,WASP36B.2025-02-17.0144_reduced.fits,WASP-36B,2025-02-18T02:16:02.269,R,150.0
6,/Users/bencapistrant/Documents/GitHub/rho_data...,WASP36B.2025-02-17.0143_reduced.fits,WASP-36B,2025-02-18T02:13:29.596,R,150.0
7,/Users/bencapistrant/Documents/GitHub/rho_data...,WASP36B.2025-02-17.0156_reduced.fits,WASP-36B,2025-02-18T02:46:44.843,R,150.0
8,/Users/bencapistrant/Documents/GitHub/rho_data...,WASP36B.2025-02-17.0173_reduced.fits,WASP-36B,2025-02-18T03:31:12.103,R,150.0
9,/Users/bencapistrant/Documents/GitHub/rho_data...,WASP36B.2025-02-17.0174_reduced.fits,WASP-36B,2025-02-18T03:33:44.574,R,150.0


In [7]:
def perform_psf_photometry(self, x, y, width, height):
        """
          Perform PSF photometry on the selected region.
          """
        # Create a dataframe to store the results
        df = pd.DataFrame(columns=['File', 'Object', 'Exptime', 'Filter', 'Star', 'X', 'Y', 'Flux', 'Error'])
        
        # Fill in the file, object, exptime, and filter information
        df['File'] = [self.frame_info['File'][self.current_index]]
        df['Object'] = [self.frame_info['Object'][self.current_index]]
        df['Exptime'] = [self.frame_info['Exptime'][self.current_index]]
        df['Filter'] = [self.frame_info['Filter'][self.current_index]]
        
        # Extract the selected region
        sub_image = self.image_data[y:y+height, x:x+width]
        
        # Perform PSF photometry on the selected region
        mean, median, std = sigma_clipped_stats(sub_image, sigma=3.0, maxiters=5)
        
        # Find the peaks in the selected region
        peaks_tbl = find_peaks(sub_image, mean + 5 * std, box_size=5).to_pandas().sort_values(by='peak_value', ascending=False).reset_index(drop=True)
        
        # Create a cutout region of the star
        star_cutout = sub_image[int(peaks_tbl['y_peak'].iloc[0]) - height:int(peaks_tbl['y_peak'].iloc[0]) + height,
                                int(peaks_tbl['x_peak'].iloc[0]) - width:int(peaks_tbl['x_peak'].iloc[0]) + width]
        
        # Calculate stats for the star cutout
        star_mean, star_median, star_std = sigma_clipped_stats(star_cutout, sigma=3.0, maxiters=5)
        
        # Calculate the flux and error
        flux = star_cutout[star_cutout > 3.*star_std].sum()
        flux_err = np.sqrt(flux)
        
        # Fill in the PSF photometry results
        df['Star'] = [1]
        df['x_peak'] = [x + peaks_tbl['x_peak'].iloc[0]]
        df['y_peak'] = [y + peaks_tbl['y_peak'].iloc[0]]
        df['Flux'] = [flux]
        df['Error'] = [flux_err]
        
        # Store the dataframe in the dictionary
        self.psf_results[self.current_index] = df
        
        # Display PSF information on the image
        self.display_psf(x, y, width, height)